In [1]:
#|default_exp process

In [13]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

# Install and load all libraries

In [14]:
#|export
from fastai.vision.all import *
from ml_utils import *
from skimage.measure import label,regionprops,find_contours
from evalutils import DetectionAlgorithm
from evalutils.validators import UniquePathIndicesValidator,DataFrameValidator
from evalutils.exceptions import ValidationError
import json, random, SimpleITK, gc, cv2
from typing import Tuple, Dict
from pandas import DataFrame
import os

In [15]:
from nbdev.export import nb_export

In [16]:
#|export
execute_in_docker = False

In [17]:
#|export
class VideoLoader():
    def load(self, *, fname):
        path = Path(fname)
        print(path)
        if not path.is_file():
            raise IOError(f"Could not load {fname} using {self.__class__.__qualname__}.")
        return [{"path": fname}]

    # only path valid
    def hash_video(self, input_video):
        pass

In [18]:
#|export
class UniqueVideoValidator(DataFrameValidator):
    """
    Validates that each video in the set is unique
    """

    def validate(self, *, df: DataFrame):
        try:
            hashes = df["video"]
        except KeyError:
            raise ValidationError("Column `video` not found in DataFrame.")

        if len(set(hashes)) != len(hashes):
            raise ValidationError("The videos are not unique, please submit a unique video for each case.")

In [19]:
v=VideoLoader()
v.load(fname='./test/input/vid_1_short.mp4')

test/input/vid_1_short.mp4


[{'path': './test/input/vid_1_short.mp4'}]

In [36]:
#|export
class Surgtoolloc_det(DetectionAlgorithm):
    def __init__(self):
        
        print(' ')
        print('TeamZERO prediction engine has started!.')

        super().__init__(
            index_key='input_video',
            file_loaders={'input_video': VideoLoader()},
            input_path=Path("/input/") if execute_in_docker else Path("./test/input/"),
            output_file=Path("/output/surgical-tool-presence.json") if execute_in_docker else Path(
                "./test/output/surgical-tool-presence.json"),
            validators=dict(input_video=(UniquePathIndicesValidator(),)),
        )
        # loading ensemble learner
        print('-Loading key artefacts...')

        ensem_path=Path('/opt/algorithm/models/cls') if execute_in_docker else Path("test/algorithm/cls")
        # segmen_path=Path('/opt/algorithm/models/seg') if execute_in_docker else Path("test/algorithm/seg")

        self.ensem_learner=[load_learner(m, cpu=False) for m in ensem_path.ls() if m.suffix=='.pkl']
        print(f'-{len(self.ensem_learner)} mutli-class classification models have been detected & loaded.')

        # self.crop_learner=load_learner(segmen_path/'seg_v1.pkl', cpu=False)
        # print('-Segmntation model for image cropping is also loaded!.')

        self.tool_list = ["needle_driver",
                          "monopolar_curved_scissor",
                          "force_bipolar",
                          "clip_applier",
                          "tip_up_fenestrated_grasper",
                          "cadiere_forceps",
                          "bipolar_forceps",
                          "vessel_sealer",
                          "suction_irrigator",
                          "bipolar_dissector",
                          "prograsp_forceps",
                          "stapler",
                          "permanent_cautery_hook_spatula",
                          "grasping_retractor"]
        print('-Tools dictionary loaded!.')
        print(' ')

    
#     def crop_images(self, src):
#         fs=get_image_files(src)
#         print(f'-Image cropping begun for {len(fs)} images...')

#         preds,_ = self.crop_learner.get_preds(dl=self.crop_learner.dls.test_dl(fs))
#         for p, f in zip(preds,self.crop_learner.dl.items):

#             fn = f.name

#             im=PILImage.create(f)
#             (h,w)=im.shape
#             mask=PILMask.create((np.array(p.argmax(0))*255).astype(np.uint8))
#             mask=Resize((h,w), ResizeMethod.Squish) (mask)

#             lbl = label(np.array(mask))
#             props = regionprops(lbl)
#             x1,y1,x2,y2=props[0].bbox[0],props[0].bbox[2],props[0].bbox[1],props[0].bbox[3]

#             im_c = PILImage.create(np.array(im)[x1:y1,x2:y2])
#             im_c.save(src/fn)
    
#         print(f'--{len(get_image_files(src))} images have been croppped. Cropping done!.')
        
    def extract_images(self, video_file):     
        
        print('-Image extraction started..')
        # start the loop
        count = 0
        src=Path('/images') if execute_in_docker else Path("./test/input/")
        
        for i in get_image_files(src): os.remove(i) 
        
        # read the video file    
        cap = cv2.VideoCapture(str(src/video_file))
        
        while True:
            is_read, f = cap.read()
            if not is_read:
                # break out of the loop if there are no frames to read
                break
            name = str(src/f'im_{count}.jpg')
            cv2.imwrite(name,f)
            count+=1
        cap.release()
        print(f'--{len(get_image_files(src))} images from {video_file} are extracted in {src} folder. Extraction done!.')
        
    def tool_detect_json_sample(self):
        # single output dict
        slice_dict = {"slice_nr": 1}
        tool_boolean_dict = {i: False for i in self.tool_list}
        single_output_dict = {**slice_dict, **tool_boolean_dict}
        return single_output_dict

    def process_case(self, *, idx, case):

        # Input video would return the collection of all frames (cap object)
        input_video_file_path = case #VideoLoader.load(case)
        # Detect and score candidates
        scored_candidates = self.predict(case.path) #video file > load evalutils.py

        # return
        # Write resulting candidates to result.json for this case
        return scored_candidates

    def save(self):
        with open(str(self._output_file), "w") as f:
            json.dump(self._case_results[0], f)


    def predict(self, fname) -> Dict:
        """
        Inputs:
        fname -> video file path
        
        Output:
        tools -> list of prediction dictionaries (per frame) in the correct format as described in documentation 
        """
        
        print(f'Processing the video file: {str(fname)} through TeamZERO prediction engine')
        print(f' ')
        self.extract_images(fname)
        print(' ')

        images_dir = Path('/images') if execute_in_docker else Path("./test/input/")
        # self.crop_images(images_dir)

        fs=get_image_files(images_dir)
        
        num_frames = len(fs)
        
        print(' ')
        print(f'-Tools presence detection started.')

        # generate output json
        all_frames_predicted_outputs = []
        all_undefined_tools=[]
        
        tta_res=[]
        prs_items=[]
        for learn in self.ensem_learner:
            # learn.dls.batch_size=32
            print('Before1: '+str(learn.dls.bs))
            learn.dls.bs=32
            print('After: '+str(learn.dls.bs))
            tta_res.append(learn.tta(dl=learn.dls.test_dl(fs)))
            if len(prs_items)<1:
                prs_items=learn.dl.items

        print(f'--Predictions from all models in the ensemble learner are obtained!.')
        tta_prs=first(zip(*tta_res))
        tta_prs+=tta_prs[:1]
        tta_prs=torch.stack(tta_prs)

        lbls=[]
        for i in range(len(c)):
            arm_preds = tta_prs[:,:,cfg(i):cfg(i+1)].mean(0);
            arm_idxs = arm_preds.argmax(dim=1)
            arm_vocab = np.array(vocab[i])
            lbls.append(arm_vocab[arm_idxs])

        for usm1,usm2,usm3,usm4,f in zip(lbls[0],lbls[1],lbls[2],lbls[3],prs_items):
            frame_dict=self.tool_detect_json_sample()
            frame_dict['slice_nr']=int(f.stem.replace("im_",""))
            frame_dict[usm1]=True if usm1 in frame_dict.keys() else all_undefined_tools.append(usm1)
            frame_dict[usm2]=True if usm2 in frame_dict.keys() else all_undefined_tools.append(usm2)
            frame_dict[usm3]=True if usm3 in frame_dict.keys() else all_undefined_tools.append(usm3)
            frame_dict[usm4]=True if usm4 in frame_dict.keys() else all_undefined_tools.append(usm4)
            frame_dict.pop("nan", None)
            frame_dict.pop("blank", None)
            frame_dict.pop("out_of_view", None)
            all_frames_predicted_outputs.append(frame_dict) 
        
        print(f'--Translation of class probabilities to tool labels is done!.')
        
        print(f'--Following tools remained unaccounted for: {set(all_undefined_tools)}. Please ensure if it is OK to skip these tools from the output.')
        tools=sorted(all_frames_predicted_outputs, key=lambda d: d['slice_nr']) 

        print(f'--Output JSON file generated & returned!.')
        
        print(' ')
        
        print(f'{fname} has been successfully processed!.')
        
        print(' ')
        return tools

In [37]:
%time pred_json=Surgtoolloc_det().predict('vid_1_short.mp4')

 
TeamZERO prediction engine has started!.
-Loading key artefacts...
-3 mutli-class classification models have been detected & loaded.
-Tools dictionary loaded!.
 
Processing the video file: vid_1_short.mp4 through TeamZERO prediction engine
 
-Image extraction started..
--60 images from vid_1_short.mp4 are extracted in test/input folder. Extraction done!.
 
 
-Tools presence detection started.
Before1: 64
After: 32


Before1: 64
After: 32


Before1: 64
After: 32


--Predictions from all models in the ensemble learner are obtained!.
--Translation of class probabilities to tool labels is done!.
--Following tools remained unaccounted for: {'nan'}. Please ensure if it is OK to skip these tools from the output.
--Output JSON file generated & returned!.
 
vid_1_short.mp4 has been successfully processed!.
 
CPU times: user 58.2 s, sys: 54 s, total: 1min 52s
Wall time: 1min 22s


In [38]:
!python -m process

-Start Time = 10:40:53
-Predicting masks for images: 0 -> 50000.
^C██████████████████--| 94.75% [1481/1563 02:13<00:07 0.0164]
Traceback (most recent call last):
  File "/home/bilal/mambaforge/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/bilal/mambaforge/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/bilal/mlworks/surgtoolloc2/process.py", line 61, in <module>
    preds,_ = learn.get_preds(dl=learn.dls.test_dl(fs[start:end]))
  File "/home/bilal/mambaforge/lib/python3.9/site-packages/fastai/learner.py", line 290, in get_preds
    self._do_epoch_validate(dl=dl)
  File "/home/bilal/mambaforge/lib/python3.9/site-packages/fastai/learner.py", line 236, in _do_epoch_validate
    with torch.no_grad(): self._with_events(self.all_batches, 'validate', CancelValidException)
  File "/home/bilal/mambaforge/lib/python3.9/site-packages/fastai/learner.py", line 193, in _with_events
    t

In [35]:
pred_json

[{'slice_nr': 0,
  'needle_driver': False,
  'monopolar_curved_scissor': True,
  'force_bipolar': False,
  'clip_applier': False,
  'tip_up_fenestrated_grasper': False,
  'cadiere_forceps': True,
  'bipolar_forceps': True,
  'vessel_sealer': False,
  'suction_irrigator': False,
  'bipolar_dissector': False,
  'prograsp_forceps': False,
  'stapler': False,
  'permanent_cautery_hook_spatula': False,
  'grasping_retractor': False},
 {'slice_nr': 1,
  'needle_driver': False,
  'monopolar_curved_scissor': True,
  'force_bipolar': False,
  'clip_applier': False,
  'tip_up_fenestrated_grasper': False,
  'cadiere_forceps': True,
  'bipolar_forceps': True,
  'vessel_sealer': False,
  'suction_irrigator': False,
  'bipolar_dissector': False,
  'prograsp_forceps': False,
  'stapler': False,
  'permanent_cautery_hook_spatula': False,
  'grasping_retractor': False},
 {'slice_nr': 2,
  'needle_driver': False,
  'monopolar_curved_scissor': True,
  'force_bipolar': False,
  'clip_applier': False,
  't

In [40]:
#|export
if __name__ == "__main__":
    Surgtoolloc_det().process()

In [41]:
nb_export('09_inference.ipynb', '.')

In [42]:
nb_export('09_inference.ipynb', '/home/bilal/mlworks/surgtoolloc2022-category-1')

In [42]:
import torch; print(torch.__version__)

1.11.0


In [43]:
import torchvision; torchvision.__version__

'0.12.0'

In [5]:
from platform import python_version

print(python_version())

3.9.13


In [12]:
from fastai.test_utils import show_install
show_install()



```text
=== Software === 
python        : 3.9.13
fastai        : 2.7.8
fastcore      : 1.5.14
fastprogress  : 1.0.2
torch         : 1.11.0
nvidia driver : 515.65
torch cuda    : 11.3 / is available
torch cudnn   : 8200 / is enabled

=== Hardware === 
nvidia gpus   : 2
torch devices : 1
  - gpu0      : NVIDIA GeForce RTX 3090

=== Environment === 
platform      : Linux-5.15.0-46-generic-x86_64-with-glibc2.31
distro        : #49~20.04.1-Ubuntu SMP Thu Aug 4 19:15:44 UTC 2022
conda env     : base
python        : /home/bilal/mambaforge/bin/python
sys.path      : /home/bilal/mlworks/surgtoolloc2
/home/bilal/mambaforge/lib/python39.zip
/home/bilal/mambaforge/lib/python3.9
/home/bilal/mambaforge/lib/python3.9/lib-dynload

/home/bilal/mambaforge/lib/python3.9/site-packages
```

Please make sure to include opening/closing ``` when you paste into forums/github to make the reports appear formatted as code sections.

Optional package(s) to enhance the diagnostics can be installed with:
pip insta

In [7]:
!pip install distro